In [1]:
import re
import torch
import transformers
import rich

model_name = "gpt2-medium"
model = transformers.AutoModelForCausalLM.from_pretrained(model_name).cuda()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please use this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: CUDA path found: /cvmfs/ai.mila.quebec/apps/x86_64/common/cuda/11.1/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA_SETUP: Detected CUDA version 111
CUDA_SETUP: Loading binary /home/mila/g/gagnonju/.main/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


In [2]:
def setup_tokenizer(model):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    # if not tokenizer.pad_token_id:
    #     assert hasattr(tokenizer, "padding_side")
    #     tokenizer.add_special_tokens({"pad_token": "<pad>"})
    #     print(tokenizer.pad_token_id)
    #     model.config.pad_token_id = tokenizer.pad_token_id
    #     model.resize_token_embeddings(len(tokenizer))
    #     tokenizer.padding_side = "left"
    tokenizer.padding_side = "left"
    
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    
    return tokenizer

tokenizer = setup_tokenizer(model)

encoded = tokenizer([
        "Generate some", 
        "Grab popcorn, and let's go",
    ], 
    return_tensors="pt", add_special_tokens=False, padding=True
)
encoded_cuda = {k: v.cuda() for k, v in encoded.items()}
outputs = model.generate(inputs=encoded_cuda["input_ids"], attention_mask=encoded_cuda["attention_mask"], max_length=100)
for output in outputs:
    decoded = tokenizer.decode(output).strip()
    cleaned = re.sub(r"\s+", " ", decoded)
    print(cleaned)

Generate logits_processor:
[]

Generate stopping critieria:
[<transformers.generation_stopping_criteria.MaxLengthCriteria object at 0x7fa16ffc8c40>]

 Doing greedy search

{
    'input_ids': tensor([[50256, 50256, 50256, 50256,  8645,   378,   617],
        [48400, 38914,    11,   290,  1309,   338,   467]], device='cuda:0'),
    'logits_processor': [],
    'stopping_criteria': [
        <transformers.generation_stopping_criteria.MaxLengthCriteria object at 
0x7fa16ffc8c40>
    ],
    'pad_token_id': 50256,
    'eos_token_id': 50256,
    'output_scores': False,
    'return_dict_in_generate': False,
    'synced_gpus': False,
    'attention_mask': tensor([[0, 0, 0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]], device='cuda:0'),
    'output_attentions': False,
    'output_hidden_states': False,
    'use_cache': None
}

<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Generate some random numbers. The following code will generate a random number between 0 and 1. var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var random = new Random(); var
Grab popcorn, and let's go. The first thing I noticed was that the movie was pretty much the same as the first one. The characters were the same, the plot was the same, and the action was the same. The only difference was that the movie was set in the future, and the characters were all teenagers. The second thing I noticed was that the movie was pretty much the same as the first one. The characters were the same, the plot was the same


In [4]:
def decode(x):
    return [re.sub(r"\s+", " ", tokenizer.decode(line)).strip() for line in x]

def generate(text, kwargs):
    encoded = tokenizer(text.strip(), return_tensors="pt").input_ids
    return model.generate(input_ids=encoded.cuda(), **kwargs)

In [5]:
gen_args = dict(
    repetition_penalty=0.5
)

outputs = generate("Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is ", gen_args)

print(outputs.shape)

print(" - \"" + "\"\n - \"".join(decode(outputs)) + "\"")

/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/transformers/generation_utils.py:1204: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 21, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


KeyboardInterrupt: 

In [ ]:
gen_args = dict(
    num_beams=5, 
    num_beam_groups=5,
    num_return_sequences=5, 
    max_length=60,
    diversity_penalty=1.,
)

outputs = generate("Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is ", gen_args)

print(outputs.shape)

print(" - \"" + "\"\n - \"".join(decode(outputs)) + "\"")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


torch.Size([5, 60])
 - "Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is (4 + 6) * 5 = (4 + 6) * 5 = (4 + 6) * 5 = (4 + 6) * 5 = (4 + 6) * 5"
 - "Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is: (4 + 6) * 5 = (4 + 6) * 2 = (4 + 6) * 1 = (4 + 6) * 0 = (4 + 6"
 - "Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is as follows: (4 + 6) * 5 = (4 + 6) * 2 = (4 + 6) * 1 = (4 + 6) * 0 = (4"
 - "Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is to add the two numbers together. The solution to the arithmetic problem ((4 + 6) * 5) - 3 is to add the two numbers together. The solution to the"
 - "Step by step, the solution to the arithmetic problem ((4 + 6) * 5) - 3 is: ((4 + 6) * 5) - 3 = 4 The solution to the arithmetic problem ((4 + 6) * 5) - 3 is: ((4"
